# Programming Assignment: 
## Готовим LDA по рецептам

Как вы уже знаете, в тематическом моделировании делается предположение о том, что для определения тематики порядок слов в документе не важен; об этом гласит гипотеза «мешка слов». Сегодня мы будем работать с несколько нестандартной для тематического моделирования коллекцией, которую можно назвать «мешком ингредиентов», потому что на состоит из рецептов блюд разных кухонь. Тематические модели ищут слова, которые часто вместе встречаются в документах, и составляют из них темы. Мы попробуем применить эту идею к рецептам и найти кулинарные «темы». Эта коллекция хороша тем, что не требует предобработки. Кроме того, эта задача достаточно наглядно иллюстрирует принцип работы тематических моделей.

Для выполнения заданий, помимо часто используемых в курсе библиотек, потребуются модули *json* и *gensim*. Первый входит в дистрибутив Anaconda, второй можно поставить командой 

*pip install gensim*

Построение модели занимает некоторое время. На ноутбуке с процессором Intel Core i7 и тактовой частотой 2400 МГц на построение одной модели уходит менее 10 минут.

### Загрузка данных

Коллекция дана в json-формате: для каждого рецепта известны его id, кухня (cuisine) и список ингредиентов, в него входящих. Загрузить данные можно с помощью модуля json (он входит в дистрибутив Anaconda):

In [1]:
import json

In [2]:
with open("recipes.json") as f:
    recipes = json.load(f)

In [3]:
print recipes[0]

{u'cuisine': u'greek', u'id': 10259, u'ingredients': [u'romaine lettuce', u'black olives', u'grape tomatoes', u'garlic', u'pepper', u'purple onion', u'seasoning', u'garbanzo beans', u'feta cheese crumbles']}


### Составление корпуса

In [4]:
from gensim import corpora, models
import numpy as np

Наша коллекция небольшая, и целиком помещается в оперативную память. Gensim может работать с такими данными и не требует их сохранения на диск в специальном формате. Для этого коллекция должна быть представлена в виде списка списков, каждый внутренний список соответствует отдельному документу и состоит из его слов. Пример коллекции из двух документов: 

[["hello", "world"], ["programming", "in", "python"]]

Преобразуем наши данные в такой формат, а затем создадим объекты corpus и dictionary, с которыми будет работать модель.

In [5]:
texts = [recipe["ingredients"] for recipe in recipes]
dictionary = corpora.Dictionary(texts)   # составляем словарь
corpus = [dictionary.doc2bow(text) for text in texts]  # составляем корпус документов

In [6]:
print texts[0]
print corpus[0], corpus[1], dictionary.token2id['pepper']

[u'romaine lettuce', u'black olives', u'grape tomatoes', u'garlic', u'pepper', u'purple onion', u'seasoning', u'garbanzo beans', u'feta cheese crumbles']
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)] [(9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1)] 5


У объекта dictionary есть полезная переменная dictionary.token2id, позволяющая находить соответствие между ингредиентами и их индексами.

In [7]:
texts = [['human', 'interface', 'computer']]
dct = corpora.Dictionary(texts)  # initialize a Dictionary
dct.add_documents([["cat", "say", "meow"], ["dog"]])  # add more document (extend the vocabulary)
dct.doc2bow(["dog", "computer", "non_existent_word"])


[(0, 1), (6, 1)]

In [8]:
print dct.token2id['computer'], dct.token2id['cat'], dct.token2id

0 3 {u'meow': 4, u'dog': 6, u'cat': 3, u'say': 5, u'computer': 0, u'human': 1, u'interface': 2}


### Обучение модели
Вам может понадобиться [документация](https://radimrehurek.com/gensim/models/ldamodel.html) LDA в gensim.

__Задание 1.__ Обучите модель LDA с 40 темами, установив количество проходов по коллекции 5 и оставив остальные параметры по умолчанию. 


Затем вызовите метод модели *show_topics*, указав количество тем 40 и количество токенов 10, и сохраните результат (топы ингредиентов в темах) в отдельную переменную. Если при вызове метода *show_topics* указать параметр *formatted=True*, то топы ингредиентов будет удобно выводить на печать, если *formatted=False*, будет удобно работать со списком программно. Выведите топы на печать, рассмотрите темы, а затем ответьте на вопрос:

Сколько раз ингредиенты "salt", "sugar", "water", "mushrooms", "chicken", "eggs" встретились среди топов-10 всех 40 тем? При ответе __не нужно__ учитывать составные ингредиенты, например, "hot water".

Передайте 6 чисел в функцию save_answers1 и загрузите сгенерированный файл в форму.

У gensim нет возможности фиксировать случайное приближение через параметры метода, но библиотека использует numpy для инициализации матриц. Поэтому, по утверждению автора библиотеки, фиксировать случайное приближение нужно командой, которая написана в следующей ячейке. __Перед строкой кода с построением модели обязательно вставляйте указанную строку фиксации random.seed.__

In [9]:
np.random.seed(76543)
# здесь код для построения модели:
lda = models.LdaModel(corpus, num_topics=40, passes=5)

In [10]:
lda.show_topics(num_topics=40, num_words=10 ,formatted=True)

[(0,
  '0.084*"104" + 0.070*"279" + 0.068*"117" + 0.065*"15" + 0.044*"29" + 0.042*"118" + 0.040*"211" + 0.039*"146" + 0.026*"32" + 0.024*"555"'),
 (1,
  '0.076*"902" + 0.065*"1124" + 0.054*"722" + 0.053*"803" + 0.052*"245" + 0.044*"333" + 0.038*"129" + 0.037*"805" + 0.036*"1099" + 0.035*"116"'),
 (2,
  '0.081*"59" + 0.054*"210" + 0.052*"306" + 0.042*"15" + 0.041*"189" + 0.039*"104" + 0.034*"600" + 0.033*"475" + 0.032*"20" + 0.029*"94"'),
 (3,
  '0.074*"207" + 0.073*"15" + 0.061*"660" + 0.047*"3" + 0.047*"11" + 0.042*"664" + 0.038*"44" + 0.037*"59" + 0.036*"241" + 0.030*"743"'),
 (4,
  '0.130*"114" + 0.127*"204" + 0.088*"15" + 0.080*"13" + 0.071*"20" + 0.050*"52" + 0.040*"47" + 0.031*"51" + 0.026*"18" + 0.025*"532"'),
 (5,
  '0.079*"307" + 0.064*"46" + 0.052*"59" + 0.039*"15" + 0.030*"388" + 0.030*"476" + 0.029*"20" + 0.028*"550" + 0.025*"827" + 0.023*"367"'),
 (6,
  '0.075*"334" + 0.074*"772" + 0.050*"1004" + 0.045*"653" + 0.044*"1637" + 0.035*"701" + 0.031*"194" + 0.026*"603" + 0.025*

In [11]:
top_words = lda.show_topics(num_topics=40, num_words=10, formatted=False)
print top_words

[(0, [('104', 0.08427994), ('279', 0.06968056), ('117', 0.068429515), ('15', 0.06473539), ('29', 0.043733444), ('118', 0.04203338), ('211', 0.040040564), ('146', 0.03916694), ('32', 0.026412722), ('555', 0.024248617)]), (1, [('902', 0.07620202), ('1124', 0.064814106), ('722', 0.054198176), ('803', 0.052642293), ('245', 0.05204196), ('333', 0.043785825), ('129', 0.037965786), ('805', 0.036994305), ('1099', 0.03593933), ('116', 0.0348662)]), (2, [('59', 0.08071749), ('210', 0.054140758), ('306', 0.051643316), ('15', 0.04211001), ('189', 0.04112334), ('104', 0.03883895), ('600', 0.03407541), ('475', 0.03254046), ('20', 0.031543504), ('94', 0.028684074)]), (3, [('207', 0.07416454), ('15', 0.07306994), ('660', 0.061058544), ('3', 0.046625227), ('11', 0.046502214), ('664', 0.04196104), ('44', 0.03767574), ('59', 0.036563423), ('241', 0.036445685), ('743', 0.030443655)]), (4, [('114', 0.12985145), ('204', 0.12687473), ('15', 0.08840965), ('13', 0.0798599), ('20', 0.07067612), ('52', 0.0499038

In [12]:
#words = [word[0] for word in words for topic, words in top_words]
w = []
for topic, words in top_words:
    print topic #, words
    for word in words:
        print word[0]
        w.append(word[0])
print w

0
104
279
117
15
29
118
211
146
32
555
1
902
1124
722
803
245
333
129
805
1099
116
2
59
210
306
15
189
104
600
475
20
94
3
207
15
660
3
11
664
44
59
241
743
4
114
204
15
13
20
52
47
51
18
532
5
307
46
59
15
388
476
20
550
827
367
6
334
772
1004
653
1637
701
194
603
1337
770
7
27
98
97
230
90
362
52
3
18
93
8
23
34
216
876
11
32
102
15
131
350
9
59
15
6
104
77
103
78
113
11
391
10
1528
280
588
1085
467
1432
2223
86
414
303
11
112
73
15
3
54
110
6
57
185
59
12
202
757
1083
192
938
481
707
607
710
331
13
228
81
44
480
15
3
229
59
347
321
14
956
44
1492
821
681
868
651
1255
1008
142
15
313
52
861
525
205
318
29
949
723
1393
16
69
143
57
469
108
104
15
327
40
12
17
557
358
44
59
315
3
1048
1025
568
495
18
714
15
397
59
795
29
857
1051
822
14
19
312
114
15
52
204
462
441
289
364
47
20
490
56
59
17
77
104
436
502
11
15
21
141
679
796
208
1108
20
5
44
892
656
22
189
260
59
395
135
15
3
9
5
711
23
373
27
15
29
99
43
52
276
916
3
24
40
15
111
44
3
346
104
18
186
398
25
448
774
447
213
1222
105
2

In [13]:
for topic, words in top_words:
    print topic, [word for word in words]
    

0 [('104', 0.08427994), ('279', 0.06968056), ('117', 0.068429515), ('15', 0.06473539), ('29', 0.043733444), ('118', 0.04203338), ('211', 0.040040564), ('146', 0.03916694), ('32', 0.026412722), ('555', 0.024248617)]
1 [('902', 0.07620202), ('1124', 0.064814106), ('722', 0.054198176), ('803', 0.052642293), ('245', 0.05204196), ('333', 0.043785825), ('129', 0.037965786), ('805', 0.036994305), ('1099', 0.03593933), ('116', 0.0348662)]
2 [('59', 0.08071749), ('210', 0.054140758), ('306', 0.051643316), ('15', 0.04211001), ('189', 0.04112334), ('104', 0.03883895), ('600', 0.03407541), ('475', 0.03254046), ('20', 0.031543504), ('94', 0.028684074)]
3 [('207', 0.07416454), ('15', 0.07306994), ('660', 0.061058544), ('3', 0.046625227), ('11', 0.046502214), ('664', 0.04196104), ('44', 0.03767574), ('59', 0.036563423), ('241', 0.036445685), ('743', 0.030443655)]
4 [('114', 0.12985145), ('204', 0.12687473), ('15', 0.08840965), ('13', 0.0798599), ('20', 0.07067612), ('52', 0.049903862), ('47', 0.04034

In [14]:
ww = [word[0] for topic, words in top_words for word in words]
print ww

['104', '279', '117', '15', '29', '118', '211', '146', '32', '555', '902', '1124', '722', '803', '245', '333', '129', '805', '1099', '116', '59', '210', '306', '15', '189', '104', '600', '475', '20', '94', '207', '15', '660', '3', '11', '664', '44', '59', '241', '743', '114', '204', '15', '13', '20', '52', '47', '51', '18', '532', '307', '46', '59', '15', '388', '476', '20', '550', '827', '367', '334', '772', '1004', '653', '1637', '701', '194', '603', '1337', '770', '27', '98', '97', '230', '90', '362', '52', '3', '18', '93', '23', '34', '216', '876', '11', '32', '102', '15', '131', '350', '59', '15', '6', '104', '77', '103', '78', '113', '11', '391', '1528', '280', '588', '1085', '467', '1432', '2223', '86', '414', '303', '112', '73', '15', '3', '54', '110', '6', '57', '185', '59', '202', '757', '1083', '192', '938', '481', '707', '607', '710', '331', '228', '81', '44', '480', '15', '3', '229', '59', '347', '321', '956', '44', '1492', '821', '681', '868', '651', '1255', '1008', '142'

In [15]:
dictionary.token2id

{u'low-sodium fat-free chicken broth': 3067,
 u'sweetened coconut': 4351,
 u'baking chocolate': 4124,
 u'egg roll wrappers': 195,
 u'bottled low sodium salsa': 6280,
 u'vegan parmesan cheese': 1712,
 u'clam sauce': 5433,
 u'mahlab': 6168,
 u'(10 oz.) frozen chopped spinach, thawed and squeezed dry': 1987,
 u'figs': 1315,
 u'caramels': 3788,
 u'broiler': 3466,
 u'jalapeno chilies': 57,
 u'(15 oz.) refried beans': 5108,
 u'brioche buns': 5186,
 u'broccoli romanesco': 5876,
 u'flaked oats': 5836,
 u'anise extract': 2055,
 u'whole wheat pastry flour': 2194,
 u'ravva': 1404,
 u'bacon': 207,
 u'millet': 3209,
 u'country crock honey spread': 4572,
 u'matcha green tea powder': 325,
 u'chopped fresh thyme': 673,
 u'chicken gravy mix': 4786,
 u'walnut oil': 3608,
 u'Kraft Slim Cut Mozzarella Cheese Slices': 6703,
 u'fresh angel hair': 3628,
 u'salsify': 2727,
 u'galangal': 910,
 u'chicken schmaltz': 2331,
 u'butter crackers': 1469,
 u'jasmine': 2031,
 u'Bisquick Baking Mix': 3919,
 u'canned jala

In [16]:
from collections import Counter
word_cnt = Counter(ww)
word_cnt['11']

6

In [17]:
print word_cnt

Counter({'15': 23, '3': 12, '59': 11, '44': 11, '29': 10, '52': 9, '20': 7, '104': 7, '11': 6, '18': 4, '27': 3, '114': 3, '47': 3, '40': 3, '5': 3, '77': 3, '346': 2, '57': 2, '194': 2, '204': 2, '312': 2, '249': 2, '32': 2, '35': 2, '79': 2, '186': 2, '6': 2, '189': 2, '43': 2, '13': 2, '143': 2, '46': 2, '9': 2, '350': 2, '216': 1, '330': 1, '214': 1, '839': 1, '213': 1, '210': 1, '211': 1, '664': 1, '660': 1, '131': 1, '215': 1, '135': 1, '495': 1, '490': 1, '166': 1, '24': 1, '76': 1, '23': 1, '196': 1, '28': 1, '289': 1, '345': 1, '938': 1, '451': 1, '280': 1, '349': 1, '68': 1, '1025': 1, '700': 1, '679': 1, '577': 1, '714': 1, '1099': 1, '711': 1, '710': 1, '651': 1, '1492': 1, '260': 1, '124': 1, '129': 1, '60': 1, '58': 1, '1528': 1, '56': 1, '51': 1, '62': 1, '53': 1, '414': 1, '532': 1, '298': 1, '54': 1, '296': 1, '145': 1, '827': 1, '821': 1, '374': 1, '822': 1, '1085': 1, '318': 1, '1083': 1, '597': 1, '313': 1, '707': 1, '1124': 1, '701': 1, '315': 1, '192': 1, '157': 1

In [18]:
words_requested = [ "salt", "sugar", "water", "mushrooms", "chicken", "eggs"]
words_counted = {}
for word in words_requested:
    idx = dictionary.token2id[word]
    words_counted[word] = word_cnt[str(idx)]
    print idx, word, word_cnt[idx] #, ww[id]
    
print words_counted

c_salt = words_counted["salt"]
c_sugar = words_counted["sugar"]
c_water = words_counted["water"]
c_mushrooms = words_counted["mushrooms"]
c_chicken = words_counted["chicken"]
c_eggs = words_counted["eggs"]

print c_salt, c_sugar, c_water, c_mushrooms, c_chicken, c_eggs
print (" ".join([str(el) for el in [c_salt, c_sugar, c_water, c_mushrooms, c_chicken, c_eggs]]))

15 salt 0
52 sugar 0
29 water 0
82 mushrooms 0
729 chicken 0
9 eggs 0
{'mushrooms': 0, 'eggs': 2, 'sugar': 9, 'water': 10, 'chicken': 1, 'salt': 23}
23 9 10 0 1 2
23 9 10 0 1 2


Почему-то не работает!
def save_answers1(c_salt, c_sugar, c_water, c_mushrooms, c_chicken, c_eggs):
    #with open("./cooking_LDA_pa_task1.txt", "w+") as fout:
    try:
        with open("/home/sigor/Develop/coursera/cmp3/w4/test", "w+") as fout:
        #fout.write(" ".join([str(el) for el in [c_salt, c_sugar, c_water, c_mushrooms, c_chicken, c_eggs]]))
            fout.write("test")
        #fout.close()
    except IOError, e:
        print "error", e

In [41]:
f = open("cooking_LDA_pa_task1.txt", "w")
f.write(" ".join([str(el) for el in [c_salt, c_sugar, c_water, c_mushrooms, c_chicken, c_eggs]]))
f.close()

In [42]:
!ls

4-1.Tematicheskoe_modelirovanie-1.pdf	gensim.ipynb
4.1.Tematicheskoye_modelirovanie_1.pdf	ldamodel_xkcd
4.2.Tematicheskoye_modelirovanie_2.pdf	ldamodel_xkcd.expElogbeta.npy
4.2.Tematicheskoye-modelirovanie_2.pdf	ldamodel_xkcd.id2word
cmp3w4pa1.pdf				ldamodel_xkcd.state
cooking_LDA_pa_task1.txt		PostnaukaPeerReview.ipynb
docword.xkcd.txt			recipes.json
edit_CookingLDA_PA.ipynb		test2
edit_CookingLDA_PA.zip			vocab.xkcd.txt


In [35]:
f = open("test2", "w")

In [36]:
print f

<open file 'test2', mode 'w' at 0x7f4f01b40b70>


In [38]:
f.write("testt")
f.close()

In [39]:
print f

<closed file 'test2', mode 'w' at 0x7f4f01b40b70>


### Фильтрация словаря
В топах тем гораздо чаще встречаются первые три рассмотренных ингредиента, чем последние три. При этом наличие в рецепте курицы, яиц и грибов яснее дает понять, что мы будем готовить, чем наличие соли, сахара и воды. Таким образом, даже в рецептах есть слова, часто встречающиеся в текстах и не несущие смысловой нагрузки, и поэтому их не желательно видеть в темах. Наиболее простой прием борьбы с такими фоновыми элементами — фильтрация словаря по частоте. Обычно словарь фильтруют с двух сторон: убирают очень редкие слова (в целях экономии памяти) и очень частые слова (в целях повышения интерпретируемости тем). Мы уберем только частые слова.

In [ ]:
import copy
dictionary2 = copy.deepcopy(dictionary)

__Задание 2.__ У объекта dictionary2 есть переменная *dfs* — это словарь, ключами которого являются id токена, а элементами — число раз, сколько слово встретилось во всей коллекции. Сохраните в отдельный список ингредиенты, которые встретились в коллекции больше 4000 раз. Вызовите метод словаря *filter_tokens*, подав в качестве первого аргумента полученный список популярных ингредиентов. Вычислите две величины: dict_size_before и dict_size_after — размер словаря до и после фильтрации.

Затем, используя новый словарь, создайте новый корпус документов, corpus2, по аналогии с тем, как это сделано в начале ноутбука. Вычислите две величины: corpus_size_before и corpus_size_after — суммарное количество ингредиентов в корпусе (для каждого документа вычислите число различных ингредиентов в нем и просуммируйте по всем документам) до и после фильтрации.

Передайте величины dict_size_before, dict_size_after, corpus_size_before, corpus_size_after в функцию save_answers2 и загрузите сгенерированный файл в форму.

In [ ]:
def save_answers2(dict_size_before, dict_size_after, corpus_size_before, corpus_size_after):
    with open("cooking_LDA_pa_task2.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in [dict_size_before, dict_size_after, corpus_size_before, corpus_size_after]]))

### Сравнение когерентностей
__Задание 3.__ Постройте еще одну модель по корпусу corpus2 и словарю dictionary2, остальные параметры оставьте такими же, как при первом построении модели. Сохраните новую модель в другую переменную (не перезаписывайте предыдущую модель). Не забудьте про фиксирование seed!

Затем воспользуйтесь методом *top_topics* модели, чтобы вычислить ее когерентность. Передайте в качестве аргумента соответствующий модели корпус. Метод вернет список кортежей (топ токенов, когерентность), отсортированных по убыванию последней. Вычислите среднюю по всем темам когерентность для каждой из двух моделей и передайте в функцию save_answers3. 

In [ ]:
def save_answers3(coherence, coherence2):
    with open("cooking_LDA_pa_task3.txt", "w") as fout:
        fout.write(" ".join(["%3f"%el for el in [coherence, coherence2]]))

Считается, что когерентность хорошо соотносится с человеческими оценками интерпретируемости тем. Поэтому на больших текстовых коллекциях когерентность обычно повышается, если убрать фоновую лексику. Однако в нашем случае этого не произошло. 

### Изучение влияния гиперпараметра alpha

В этом разделе мы будем работать со второй моделью, то есть той, которая построена по сокращенному корпусу. 

Пока что мы посмотрели только на матрицу темы-слова, теперь давайте посмотрим на матрицу темы-документы. Выведите темы для нулевого (или любого другого) документа из корпуса, воспользовавшись методом *get_document_topics* второй модели:

Также выведите содержимое переменной *.alpha* второй модели:

У вас должно получиться, что документ характеризуется небольшим числом тем. Попробуем поменять гиперпараметр alpha, задающий априорное распределение Дирихле для распределений тем в документах.

__Задание 4.__ Обучите третью модель: используйте сокращенный корпус (corpus2 и dictionary2) и установите параметр __alpha=1__, passes=5. Не забудьте про фиксацию seed! Выведите темы новой модели для нулевого документа; должно получиться, что распределение над множеством тем практически равномерное. Чтобы убедиться в том, что во второй модели документы описываются гораздо более разреженными распределениями, чем в третьей, посчитайте суммарное количество элементов, __превосходящих 0.01__, в матрицах темы-документы обеих моделей. Другими словами, запросите темы  модели для каждого документа с параметром *minimum_probability=0.01* и просуммируйте число элементов в получаемых массивах. Передайте две суммы (сначала для модели с alpha по умолчанию, затем для модели в alpha=1) в функцию save_answers4.

In [ ]:
def save_answers4(count_model2, count_model3):
    with open("cooking_LDA_pa_task4.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in [count_model2, count_model3]]))

Таким образом, гиперпараметр __alpha__ влияет на разреженность распределений тем в документах. Аналогично гиперпараметр __eta__ влияет на разреженность распределений слов в темах.

### LDA как способ понижения размерности
Иногда, распределения над темами, найденные с помощью LDA, добавляют в матрицу объекты-признаки как дополнительные, семантические, признаки, и это может улучшить качество решения задачи. Для простоты давайте просто обучим классификатор рецептов на кухни на признаках, полученных из LDA, и измерим точность (accuracy).

__Задание 5.__ Используйте модель, построенную по сокращенной выборке с alpha по умолчанию (вторую модель). Составьте матрицу $\Theta = p(t|d)$ вероятностей тем в документах; вы можете использовать тот же метод get_document_topics, а также вектор правильных ответов y (в том же порядке, в котором рецепты идут в переменной recipes). Создайте объект RandomForestClassifier со 100 деревьями, с помощью функции cross_val_score вычислите среднюю accuracy по трем фолдам (перемешивать данные не нужно) и передайте в функцию save_answers5.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score

In [ ]:
def save_answers5(accuracy):
     with open("cooking_LDA_pa_task5.txt", "w") as fout:
        fout.write(str(accuracy))

Для такого большого количества классов это неплохая точность. Вы можете попроовать обучать RandomForest на исходной матрице частот слов, имеющей значительно большую размерность, и увидеть, что accuracy увеличивается на 10–15%. Таким образом, LDA собрал не всю, но достаточно большую часть информации из выборки, в матрице низкого ранга.

### LDA — вероятностная модель
Матричное разложение, использующееся в LDA, интерпретируется как следующий процесс генерации документов.

Для документа $d$ длины $n_d$:
1. Из априорного распределения Дирихле с параметром alpha сгенерировать распределение над множеством тем: $\theta_d \sim Dirichlet(\alpha)$
1. Для каждого слова $w = 1, \dots, n_d$:
    1. Сгенерировать тему из дискретного распределения $t \sim \theta_{d}$
    1. Сгенерировать слово из дискретного распределения $w \sim \phi_{t}$.
    
Подробнее об этом в [Википедии](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation).

В контексте нашей задачи получается, что, используя данный генеративный процесс, можно создавать новые рецепты. Вы можете передать в функцию модель и число ингредиентов и сгенерировать рецепт :)

In [ ]:
def generate_recipe(model, num_ingredients):
    theta = np.random.dirichlet(model.alpha)
    for i in range(num_ingredients):
        t = np.random.choice(np.arange(model.num_topics), p=theta)
        topic = model.show_topic(t, topn=model.num_terms)
        topic_distr = [x[1] for x in topic]
        terms = [x[0] for x in topic]
        w = np.random.choice(terms, p=topic_distr)
        print w

### Интерпретация построенной модели
Вы можете рассмотреть топы ингредиентов каждой темы. Большиснтво тем сами по себе похожи на рецепты; в некоторых собираются продукты одного вида, например, свежие фрукты или разные виды сыра.

Попробуем эмпирически соотнести наши темы с национальными кухнями (cuisine). Построим матрицу $A$ размера темы $x$ кухни, ее элементы $a_{tc}$ — суммы $p(t|d)$ по всем документам $d$, которые отнесены к кухне $c$. Нормируем матрицу на частоты рецептов по разным кухням, чтобы избежать дисбаланса между кухнями. Следующая функция получает на вход объект модели, объект корпуса и исходные данные и возвращает нормированную матрицу $A$. Ее удобно визуализировать с помощью seaborn.

In [ ]:
import pandas
import seaborn
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
def compute_topic_cuisine_matrix(model, corpus, recipes):
    # составляем вектор целевых признаков
    targets = list(set([recipe["cuisine"] for recipe in recipes]))
    # составляем матрицу
    tc_matrix = pandas.DataFrame(data=np.zeros((model.num_topics, len(targets))), columns=targets)
    for recipe, bow in zip(recipes, corpus):
        recipe_topic = model.get_document_topics(bow)
        for t, prob in recipe_topic:
            tc_matrix[recipe["cuisine"]][t] += prob
    # нормируем матрицу
    target_sums = pandas.DataFrame(data=np.zeros((1, len(targets))), columns=targets)
    for recipe in recipes:
        target_sums[recipe["cuisine"]] += 1
    return pandas.DataFrame(tc_matrix.values/target_sums.values, columns=tc_matrix.columns)

In [ ]:
def plot_matrix(tc_matrix):
    plt.figure(figsize=(10, 10))
    seaborn.heatmap(tc_matrix, square=True)

In [ ]:
# Визуализируйте матрицу


Чем темнее квадрат в матрице, тем больше связь этой темы с данной кухней. Мы видим, что у нас есть темы, которые связаны с несколькими кухнями. Такие темы показывают набор ингредиентов, которые популярны в кухнях нескольких народов, то есть указывают на схожесть кухонь этих народов. Некоторые темы распределены по всем кухням равномерно, они показывают наборы продуктов, которые часто используются в кулинарии всех стран. 

Жаль, что в датасете нет названий рецептов, иначе темы было бы проще интерпретировать...

### Заключение
В этом задании вы построили несколько моделей LDA, посмотрели, на что влияют гиперпараметры модели и как можно использовать построенную модель. 